# Visualize a particular experiment

In [ ]:
import logging
logging.getLogger("param.Dimension").setLevel(logging.CRITICAL)
logging.getLogger("param.ParameterizedMetaclass").setLevel(logging.CRITICAL)
logging.getLogger("param.SpreadPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.CurvePlot").setLevel(logging.CRITICAL)
logging.getLogger("param.AdjointLayout").setLevel(logging.CRITICAL)
logging.getLogger("param.HoloMap").setLevel(logging.CRITICAL)
logging.getLogger("param.OverlayPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.BarPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.ErrorPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.RasterPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.Layout").setLevel(logging.CRITICAL)
logging.getLogger("param.PointPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.DynamicMap").setLevel(logging.CRITICAL)
logging.getLogger("param.Callable").setLevel(logging.CRITICAL)
logging.getLogger("param.Points").setLevel(logging.CRITICAL)
logging.getLogger("param.Overlay").setLevel(logging.CRITICAL)
logging.getLogger("param.Curve").setLevel(logging.CRITICAL)
import importlib

import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.resources import INLINE

import functions_bondjango as bd
import functions_kinematic as fk
import functions_misc as fm
import functions_plotting as fplot
import pandas as pd
import numpy as np
import h5py
from sklearn.decomposition import PCA
import processing_parameters


In [ ]:
# Load the desired files
importlib.reload(processing_parameters)
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = processing_parameters.vis_string
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if 'preproc' in el['slug']]
print(data_path)


In [ ]:
# Plot the raw trajectories
# define the target file
target_file = 0
# load the data
with pd.HDFStore(data_path[target_file]) as h:
    variables = h['full_traces']
    if '/matched_calcium' in h.keys():
        calcium = h['matched_calcium']    
# get the target column names
# column_list_x = [el for el in variables.columns if ('mouse' in el) and ('_x' in el)]
# column_list_y = [el for el in variables.columns if ('mouse' in el) and ('_y' in el)]

column_list_x = [el for el in variables.columns if ('_x' in el)]
column_list_y = [el for el in variables.columns if ('_y' in el)]

# for all the columns
for idx, el in enumerate(column_list_x):
    fplot.plot_2d([[variables.loc[:, el]]])
#     fplot.plot_2d([[variables.loc[:, [el, column_list_y[idx]]].to_numpy()]])

In [ ]:
# Load a particular file

# define the target file
target_file = 0
# load the data
with pd.HDFStore(data_path[target_file]) as h:
    variables = h['full_traces']
    if '/matched_calcium' in h.keys():
        calcium = h['matched_calcium']    
    print(data_path[target_file])


# Plot data without calcium
hv.extension('bokeh')
def mouse_trajectory(time):
    x = variables.loc[time, ['mouse_barl_x', 'mouse_barr_x', 
                             'mouse_head_x', 'mouse_x', 'mouse_body2_x', 
                                'mouse_body3_x', 'mouse_base_x']].to_numpy()
    y = variables.loc[time, ['mouse_barl_y', 'mouse_barr_y',
                             'mouse_head_y', 'mouse_y', 'mouse_body2_y', 
                                'mouse_body3_y', 'mouse_base_y']].to_numpy()
    
    heading = variables.mouse_heading[time]
    arrow_head = hv.Points((x[0], y[0])).opts(marker='triangle', color='black', angle=heading-90, size=10)
    arrow_body = hv.Points((x[1], y[1])).opts(marker='dash', color='black', angle=heading, size=20)

    return hv.Curve((x, y))*arrow_head*arrow_body

def cricket_trajectory(time):
    x = variables.loc[time, ['cricket_0_head_x', 'cricket_0_x']].to_numpy()
    y = variables.loc[time, ['cricket_0_head_y', 'cricket_0_y']].to_numpy()
    heading = variables.mouse_heading[time]
    arrow_head = hv.Points((x[0], y[0])).opts(marker='triangle', color='green', angle=heading-90, size=10)
    arrow_body = hv.Points((x[1], y[1])).opts(marker='dash', color='green', angle=heading, size=20)
    
    return hv.Curve((x, y))*arrow_head*arrow_body

mouse_map = hv.DynamicMap(mouse_trajectory, kdims=['time'])

cricket_map = hv.DynamicMap(cricket_trajectory, kdims=['time'])


both_map = (mouse_map*cricket_map).opts(width=600, height=600, xlim=(-5, 45), ylim=(-5, 45))
both_panel = pn.panel(both_map.redim.range(time=(0, variables.shape[0])), center=True, widget_location='top')
both_panel

In [ ]:
# plot the data with calcium
hv.extension('bokeh')

def mouse_trajectory(time):
    x = calcium.loc[:time+1, 'mouse_x'].to_numpy()
    y = calcium.loc[:time+1, 'mouse_y'].to_numpy()
    
    labels = list(calcium.columns)
    cells = [el for el in labels if 'cell' in el]
    calcium_data = calcium.loc[time, cells].to_numpy()
    if np.all(np.isnan(calcium_data)):
        calcium_data = 0
    else:
        calcium_data = np.nanmean(calcium_data)
        
    activity = hv.Points((x[-1], y[-1])).opts(marker='circle', color='red', size=calcium_data*1000)
    
    heading = calcium.mouse_heading[time]
    arrow_head = hv.Points((x[-1], y[-1])).opts(marker='triangle', color='black', angle=heading-90, size=10)
    arrow_body = hv.Points((x[-2], y[-2])).opts(marker='dash', color='black', angle=heading, size=20)

    return hv.Curve((x, y))*arrow_head*arrow_body*activity

def cricket_trajectory(time):
    x = calcium.loc[:time+1, 'cricket_x'].to_numpy()
    y = calcium.loc[:time+1, 'cricket_y'].to_numpy()
    heading = calcium.mouse_heading[time]
    arrow_head = hv.Points((x[-1], y[-1])).opts(marker='triangle', color='green', angle=heading-90, size=10)
    arrow_body = hv.Points((x[-2], y[-2])).opts(marker='dash', color='green', angle=heading, size=20)
    
    return hv.Curve((x, y))*arrow_head*arrow_body

mouse_map = hv.DynamicMap(mouse_trajectory, kdims=['time'])

cricket_map = hv.DynamicMap(cricket_trajectory, kdims=['time'])


both_map = (mouse_map*cricket_map).opts(width=600, height=400, xlim=(0, 1280), ylim=(0, 1024))
both_panel = pn.panel(both_map.redim.range(time=(0, calcium.shape[0])), center=True, widget_location='top')
both_panel

In [ ]:
# show the PCA time trajectory of the neural activity

hv.extension('plotly')
# get the calcium data for this experiment
labels = list(calcium.columns)
cells = [el for el in labels if 'cell' in el]
calcium_data = calcium.loc[:, cells].to_numpy()
# create the PCA object
pca = PCA(n_components=3)
# reduce the dimensionality of the neural activity over time
reduced_ca = pca.fit_transform(calcium_data)

# plot the trajectory
curve3d = hv.Path3D(reduced_ca)*hv.Scatter3D(np.reshape(reduced_ca[0,:],(1,-1)))
curve3d

In [ ]:
# Display the data with video

